In [11]:
import pandas as pd
import numpy as np

from csh_fantasy_bot import bot
from csh_fantasy_bot.roster_change_optimizer import RosterChangeSet
from csh_fantasy_bot.nhl import score_team

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [12]:
# true value ignores actuals if they exist and works off projections only
simulation_mode=False
week_number = 13
l_id = '419.l.90115'
# league_id = "403.l.18782"
manager = bot.ManagerBot(league_id=l_id, week=week_number, simulation_mode=simulation_mode)
league_scorer = manager.game_week().score_comparer
opp = manager.tm.matchup(week_number)
print(f'Opponent: {manager._get_team_name(manager.lg, opp)}')

INFO:root:transactions.pkl file is stale.  Expired at 2023-01-05 08:24:50.003424
INFO:root:Building new transactions.pkl file
INFO:root:Finished building transactions.pkl file


Opponent: Vancouver Koho's


In [13]:
# paste roster change text into here
"""
Here is example of what roster change text should look like
It is output from the rosterchangeset.pretty_print method

Date: 2021-02-08, in: Alexis Lafreniere(8641), out: Teuvo Teravainen(5698)
Date: 2021-02-09, in: Zdeno Chara(1700), out: Kailer Yamamoto(7537)
Date: 2021-02-10, in: Adam Fox(7174), out: Evander Kane(4684)
"""
roster_change_text="""
"""

roster_changes:RosterChangeSet = RosterChangeSet.from_pretty_print_text(roster_change_text, manager.game_week(week_number).all_player_predictions)

scores_with_changes = manager.score_team(roster_change_set=roster_changes)[1]
scoring_result = league_scorer.score(scores_with_changes)[manager.stat_categories]
# print(scoring_result)
# print(f"\nScore: opp->({scoring_result.loc['score_opp'].sum()}), league->({scoring_result.loc['score_league'].sum()})") 
if (roster_change_text != '\n'):
    scores_ignore_roster_changes = manager.score_team()[1]
    score_result_no_changes = league_scorer.score(scores_ignore_roster_changes)
    print (f"\nWithout roster changes: opp->({score_result_no_changes.loc['score_opp'].sum()}), league->({score_result_no_changes.loc['score_league'].sum()})") 

In [14]:
scores_ignore_roster_changes = scores_with_changes
games_played = scores_ignore_roster_changes['G'].groupby(level=1).count()
result = manager.game_week().all_player_predictions.copy()
result['GP'] = games_played
gp_table = result[result.GP.notna()].sort_values('GP', ascending=False).round(2)
gp_table[['name', 'GP', 'fpts']]

,name,GP,fpts
player_id,,,
4351,Brad Marchand,4.0,1.61
6367,Aaron Ekblad,3.0,0.94
4471,Steven Stamkos,3.0,1.55
5431,Vincent Trocheck,3.0,1.27
6368,Sam Reinhart,3.0,1.47
6908,Andrew Mangiapane,3.0,0.38
6877,Kirill Kaprizov,3.0,2.43
6762,Joel Eriksson Ek,3.0,1.15
5986,Darnell Nurse,3.0,0.73


In [21]:
gp_table['GP'].sum()


42.0

In [22]:
def display_sort_ftps(df, ascending=False):
    return df.sort_values('fpts', ascending=ascending)[['name','eligible_positions', 'status','fantasy_status','fpts']]

In [23]:
rest_season_predictions = manager.game_week().all_player_predictions

In [24]:
# manager.game_week().all_player_predictions.head()
my_team = rest_season_predictions[rest_season_predictions.fantasy_status == 8]
display_sort_ftps(my_team)


,name,eligible_positions,status,fantasy_status,fpts
player_id,,,,,
5376,J.T. Miller,"[C, LW, RW]",,8,1.558006
7902,Brady Tkachuk,[LW],,8,1.553602
6369,Leon Draisaitl,"[C, LW]",,8,1.389591
4976,Vladimir Tarasenko,[RW],,8,1.045964
5992,Josh Morrissey,[D],,8,0.918164
5697,Tomas Hertl,[C],,8,0.764111
4967,Jeff Skinner,[LW],,8,0.696263
3980,Jordan Staal,[C],,8,0.687850
5769,Shayne Gostisbehere,[D],,8,0.382698


In [25]:

avail_d_selector = rest_season_predictions.eligible_positions.map(set(['D']).issubset) 
fas = rest_season_predictions.fantasy_status == 'FA'
fas
predicted_d = rest_season_predictions[(avail_d_selector) & (fas)]
display_sort_ftps(predicted_d)
# rest_season_predictions['fantasy_status'].value_counts()
manager.game_week().all_player_predictions.loc[[7910,7908,4686],['name', 'G','A','SOG', 'HIT']]


,name,G,A,SOG,HIT
player_id,,,,,
7910,Noah Dobson,0.19,0.46,2.37,0.84
7908,Evan Bouchard,0.11,0.31,1.97,0.93
4686,Oliver Ekman-Larsson,0.07,0.30,2.03,1.57


In [26]:
display_sort_ftps(rest_season_predictions).head(30)


,name,eligible_positions,status,fantasy_status,fpts
player_id,,,,,
7109,Auston Matthews,[C],,5,2.444334
6743,Connor McDavid,[C],,3,2.403382
6752,Mikko Rantanen,"[C, RW]",,4,2.245891
6877,Kirill Kaprizov,[LW],,7,2.205879
7114,Matthew Tkachuk,[RW],,2,2.139115
3637,Alex Ovechkin,[LW],,9,2.128135
7519,Cale Makar,[D],,11,2.060420
5980,Nathan MacKinnon,"[C, IR]",IR,12,1.999369
7520,Elias Pettersson,"[C, LW]",,9,1.967469
